In [ ]:
Aurora Forecasting - Part 02: Daily Feature Pipeline

🗒️ This notebook is divided into the following sections:
Initialize Hopsworks connection.

Fetch the latest real-time Solar Wind data from NOAA.

Fetch the latest Cloud Cover forecast for Stockholm, Luleå, and Kiruna.

Update the Feature Groups in the Hopsworks Feature Store.

📝 Imports and Login

In [ ]:
import pandas as pd
import datetime
import hopsworks
from config import HopsworksSettings
import util
import warnings
warnings.filterwarnings("ignore")

# Setup settings
settings = HopsworksSettings()

# Login to Hopsworks
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY.get_secret_value()
)
fs = project.get_feature_store()

🛰️ Step 1: Get Real-time Solar Wind Data

We use the NOAA SWPC API to get the most recent measurements from the DSCOVR/ACE satellites. These will serve as the features for our real-time inference.

In [ ]:
print("Fetching real-time solar wind data from NOAA...")

# Uses the helper function from util.py to fetch and merge mag/plasma data
new_solar_df = util.get_noaa_realtime_data(
    settings.NOAA_MAG_URL,
    settings.NOAA_PLASMA_URL
)

# Format the time_tag for Hopsworks compatibility
new_solar_df['time_tag'] = new_solar_df['time_tag'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(f"Successfully retrieved {len(new_solar_df)} new solar wind records.")
new_solar_df.head()

☁️ Step 2: Get Daily Weather Forecast

To decide if the aurora is "Visible," we need the cloud cover forecast for our three target cities.

In [ ]:
weather_data = []
today = datetime.date.today().strftime('%Y-%m-%d')

for city, coords in settings.CITIES.items():
    print(f"Fetching cloud cover forecast for {city}...")

    # Get current cloud cover percentage from Open-Meteo
    cloud_cover = util.get_city_weather_forecast(coords['lat'], coords['lon'])

    weather_data.append({
        'city': city,
        'date': today,
        'cloud_cover': cloud_cover
    })

new_weather_df = pd.DataFrame(weather_data)
new_weather_df

⬆️ Step 3: Insert into Feature Groups

Now we push the new observations into the Feature Store. Hopsworks will handle the deduplication based on the primary keys defined in the backfill notebook.

In [ ]:
# Retrieve references to the Feature Groups
solar_wind_fg = fs.get_feature_group(name="solar_wind_fg", version=1)
city_weather_fg = fs.get_feature_group(name="city_weather_fg", version=1)

# Insert new data
# Note: For real-time pipelines, we often use online_enabled=True
# so the data is available for immediate inference.
solar_wind_fg.insert(new_solar_df)
city_weather_fg.insert(new_weather_df)

print("Daily Feature Pipeline execution complete!")